# 5 Department Grant Import Pipeline

#### Global Variables

In [1]:
LIMIT_TO_NEW_ENTRIES_ONLY = True
REMOVE_STATE_GOVTS = True

#### Library Imports

In [2]:
import pandas as pd
import os
import re
import glob
import warnings
from datetime import datetime
import json

pd.options.display.max_columns=200
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

#### Data Import & Filtering

In [3]:
file_paths = glob.glob('usa_spending_contract_data/indiv_dpt_dta/**/*.csv', recursive=True)
all_data = [pd.read_csv(g) for g in file_paths]
df = pd.concat(all_data)
del all_data

In [4]:
clean_df = df.copy()

if LIMIT_TO_NEW_ENTRIES_ONLY:
    clean_df = clean_df[clean_df['action_type_description'] == 'NEW']

if REMOVE_STATE_GOVTS:
    clean_df = clean_df[clean_df['business_types_description'] != 'STATE GOVERNMENT']

In [5]:
# De-dupe grants
dup_cols = [
    'usaspending_permalink',
]

clean_df.sort_values(by='action_date', ascending=True, inplace=True)

clean_df.drop_duplicates(subset=dup_cols, keep='last', inplace=True)

#### Basic Data Descriptions

In [6]:
print(len(clean_df))
clean_df.head(2)

361584


,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,non_federal_funding_amount,total_non_federal_funding_amount,face_value_of_loan,original_loan_subsidy_cost,total_face_value_of_loan,total_loan_subsidy_cost,generated_pragmatic_obligations,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_from_COVID-19_supplementals_for_overall_award,obligated_amount_from_COVID-19_supplementals_for_overall_award,outlayed_amount_from_IIJA_supplemental_for_overall_award,obligated_amount_from_IIJA_supplemental_for_overall_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,recipient_uei,recipient_duns,recipient_name,recipient_name_raw,recipient_parent_uei,recipient_parent_duns,recipient_parent_name,recipient_parent_name_raw,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_code,recipient_city_name,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_recipient_state_fips_code,recipient_state_code,recipient_state_name,recipient_zip_code,recipient_zip_last_4_code,prime_award_transaction_recipient_cd_original,prime_award_transaction_recipient_cd_current,recipient_foreign_city_name,recipient_foreign_province_name,recipient_foreign_postal_code,primary_place_of_performance_scope,primary_place_of_performance_country_code,primary_place_of_performance_country_name,primary_place_of_performance_code,primary_place_of_performance_city_name,prime_award_transaction_place_of_performance_county_fips_code,primary_place_of_performance_county_name,prime_award_transaction_place_of_performance_state_fips_code,primary_place_of_performance_state_name,primary_place_of_performance_zip_4,prime_award_transaction_place_of_performance_cd_original,prime_award_transaction_place_of_performance_cd_current,primary_place_of_performance_foreign_location,cfda_number,cfda_title,funding_opportunity_number,funding_opportunity_goals_text,assistance_type_code,assistance_type_description,transaction_description,prime_award_base_transaction_description,business_funds_indicator_code,business_funds_indicator_description,business_types_code,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
3529,1650_CS31025CS9_CS31025CS9-00-000001_17.504_00,ASST_NON_CS31025CS9_1650,CS31025CS9,0,CS31025CS9-04-000001,SAI EXEMPT,58000.0,761752.74,761752.74,NaN,0.0,0.0,0.0,0.0,0.0,0.0,58000.0,NaN,NaN,NaN,NaN,NaN,2018-10-01,2019,2018-10-01,2019-09-30,1601,Department of Labor,1650,Occupational Safety and Health Administration,1650GR,U.S. DEPARTMENT OF LABOR,1601.0,Department of Labor,1650.0,Occupational Safety and Health Administration,1650CE,"DEPT OF LABOR, OSHA COMPLIANCE ASST",016-2019/2019-0400-000,016-0400,25.2: Other services from non-Federal sources,0006: COMPLIANCE ASSISTANCE - STATE;0006: FEDE...,NTXJM52SHKS7,7

In [7]:
clean_df[['awarding_agency_name', 'action_date_fiscal_year']].value_counts().unstack()

action_date_fiscal_year,2019,2020,2021,2022,2023,2024
awarding_agency_name,,,,,,
Department of Education,7399,20514,10715,9997,8922,1497
Department of Health and Human Services,21745,30558,35594,24299,27107,7908
Department of Housing and Urban Development,20842,29754,22173,19617,21719,16178
Department of Justice,2957,4377,1963,6521,5026,192
Department of Labor,711,645,671,628,1007,348


#### Data Cleaning & Processing

In [8]:
clean_df['total_outlayed_amount_for_overall_award'] = clean_df['total_outlayed_amount_for_overall_award'].fillna(0)

clean_df['estimated_remaining_funds'] = clean_df['total_obligated_amount'] - clean_df['total_outlayed_amount_for_overall_award']

clean_df['spent_percent'] = ((clean_df['total_outlayed_amount_for_overall_award'] / clean_df['total_obligated_amount'])*100).round(0)

clean_df['remaining_funds_percent'] = ((clean_df['estimated_remaining_funds'] / clean_df['total_obligated_amount'])*100).round(0)

clean_df['period_of_performance_current_end_date'] = pd.to_datetime(clean_df['period_of_performance_current_end_date'])
clean_df['period_of_performance_start_date'] = pd.to_datetime(clean_df['period_of_performance_start_date'])
clean_df['action_date'] = pd.to_datetime(clean_df['action_date'])

clean_df['grant_is_open'] = clean_df['period_of_performance_current_end_date'] >= datetime(2024,9,1)

#### Understanding description values

In [9]:
# This is really just used to match descriptions.
# Writing to csv files is currently surpressed.
def get_description_counts_by_agency(df, desc_col):
    counts_df = df[[desc_col, 'awarding_agency_name']].value_counts().reset_index(name='count')
    counts_df[desc_col] = counts_df[desc_col].str.split(';')
    exploded_df = counts_df.explode(desc_col)
    return exploded_df.groupby([desc_col, 'awarding_agency_name'])['count'].sum().reset_index()

    
# get_description_counts_by_agency(clean_df, 'cfda_title').to_csv('programs_cfda.csv', index=False)
# get_description_counts_by_agency(clean_df, 'program_activities_funding_this_award').to_csv('programs.csv', index=False)

In [10]:
with open('program_mapping.json', 'r') as file:
    mappings = json.load(file)

In [11]:
missing_values = pd.DataFrame(columns=['Department', 'Missing Program Mapping Name'])

# Iterate over the JSON data
for top_key, top_value in mappings.items():
    for second_key, second_value in top_value.items():
        # Check if specific sub-keys exist and are both empty arrays
        if (second_value.get('cfda_descriptions') == [] and 
            second_value.get('program_activities_funding') == []):
            # Append to the DataFrame
            missing_values = missing_values.append({'Department': top_key, 
                                                    'Missing Program Mapping Name': second_key}, ignore_index=True)

missing_values

,Department,Missing Program Mapping Name
0,Department of Education,Project Parent
1,Department of Education,Stronger Connections
2,Department of Education,Transitioning Gang-Involved Youth to Higher Ed...
3,Department of Labor,Growth Opportunities - Youth Program
4,Department of Labor,Pathways Home - Employment Program
5,Department of Health and Human Services,NIH Firearm Injury and Mortality Research
6,Department of Health and Human Services,CDC Preventing Violence Affecting Young Lives
7,Department of Health and Human Services,CDC National Centers for Excellence in Youth V...
8,Department of Justice,Hospital Based Victim Services
9,Department of Justice,Center For Culturally Responsive Victim Services


#### Some notes on missing categories...

- DOJ Youth Violence and Reductions could be "0005: YOUTH VIOLENCE PREVENTION"
- Matches for "Community Development and Block Grants" from HUD are a bit questionable at the moment.
- For something like NIH Firearm Injury and Mortality Research, we may need to get down to the sub-funder level and then use the "0015: INJURY PREVENTION AND CONTROL (0952)" program, or something like that.
- Do we want any of "CRIME VICTIM ASSISTANCE," "CRIME VICTIM ASSISTANCE/DISCRETIONARY GRANTS," or "CRIME VICTIM COMPENSATION" as cfda categories for VOCA?
- Growth Opportunities - Youth Program was just started in 2024: https://www.dol.gov/newsroom/releases/eta/eta20240207
- Pathway Home seems to be the same thing as a re-entry funding program (based on text in description fields)
- Summer employment opportunities for youth from Labor seem to need to be derived from description fields. There's no consistent program or fund they come from. (Well, Youth Programs is a lot of them, but those are going to be a pretty mixed bag.)


#### Some notes on categories in general...

- What do we do when we have multiple matches for one grant?
- I made only a couple of changes to DOJ info:
    - Added "COMMUNITY-BASED VIOLENCE INTERVENTION AND PREVENTION INITIATIVE" as a cfda description
    - Added "STOP SCHOOL VIOLENCE" as a cfda description
    - Added "SECOND CHANCE ACT REENTRY INITIATIVE" as a cfda description

In [12]:
def find_matches(row, mappings):
    office_name = row['awarding_agency_name']
    cfda_matches = []
    program_funding_matches = []
    
    # Create a dictionary to track the order of appearance
    match_order = {}
    order_counter = 0
    
    # Get the associated key in mappings
    if office_name in mappings:
        programs = mappings[office_name]
        
        # Loop through each sub-key in programs
        for prog_key, prog in programs.items():
            # Check for matches in cfda_title
            if any(description in row['cfda_title'] for description in prog['cfda_descriptions']):
                cfda_matches.append(prog_key)
                if prog_key not in match_order:
                    match_order[prog_key] = order_counter
                    order_counter += 1
                
            # Check for matches in program_activities_funding_this_award
            if pd.notna(row['program_activities_funding_this_award']) and any(activity in row['program_activities_funding_this_award'] for activity in prog['program_activities_funding']):
                program_funding_matches.append(prog_key)
                if prog_key not in match_order:
                    match_order[prog_key] = order_counter
                    order_counter += 1

    # Sort matches based on their first appearance
    cfda_matches.sort(key=lambda x: match_order[x])
    program_funding_matches.sort(key=lambda x: match_order[x])
    
    match_on_program = pd.NA
    if len(program_funding_matches) > 0:
        match_on_program = True
    elif len(cfda_matches) > 0:
        match_on_program = False
    
    # Get unique matches in the order of their appearance
    unique_matches = list(dict.fromkeys(program_funding_matches + cfda_matches))
    
    # Add the matches and their count to new columns
    row['prog_matches'] = unique_matches
    row['n_prog_matches'] = len(unique_matches)
    row['matched_on_program'] = match_on_program
    # Prioritizes program_activities_funding_this award then cfda_title, first in list for either one
    row['program_match'] = unique_matches[0] if len(unique_matches) > 0 else pd.NA
    
    return row


clean_df = clean_df.apply(find_matches, axis=1, mappings=mappings)

In [13]:
pd.DataFrame(clean_df[[
    'awarding_agency_name',
    'program_match'
]].value_counts(dropna=False)).sort_index(level=0)

0
awarding_agency_name                        program_match                                             
Department of Education                     21st Century Learning Centers                           29
                                            Education Innovation and Research Program              146
                                            Full Service Community Schools                          79
                                            Promise Neighborhoods                                  384
                                            Student Support and Academic Enrichment                 38
                                            NaN                                                  58368
Department of Health and Human Services     NaN                                                 147211
Department of Housing and Urban Development Choice Neighborhoods                                   100
                                            Community Development and Block Grants               11995
                                            NaN                                                 118188
Department of Justice                       Byrne Criminal Justice Innovation                       48
                                            Byrne Memorial Justice Assistance Grant               4803
                                            Community Policing Development Micro-Grants            592
                                            Comunity Violence Intervention And Prevention I...     155
                                            Cops Hiring Program                                    852
                                            Project Safe Neighborhood                              173
                                            School Violence Prevention Program                     473
                                            Second Chance Act Community Based Re-Entry Program     344
                                            Smart Policing                                          31
                                            Strategies To Support Children Exposed To Violence      40
                                            Victims of Crime Act (VOCA)                           1211
                                            NaN                                                  12314
Department of Labor                         Youth Build                                            424
                                            NaN                                                   3586

In [14]:
clean_df.n_prog_matches.value_counts(dropna=False)

0    339667
1     21395
2       522
Name: n_prog_matches, dtype: int64

In [15]:
clean_df[[
    'awarding_agency_name',
    'n_prog_matches'
]].value_counts(dropna=False).unstack()

n_prog_matches,0,1,2
awarding_agency_name,,,
Department of Education,58368.0,200.0,476.0
Department of Health and Human Services,147211.0,NaN,NaN
Department of Housing and Urban Development,118188.0,12095.0,NaN
Department of Justice,12314.0,8676.0,46.0
Department of Labor,3586.0,424.0,NaN


In [16]:
clean_df.matched_on_program.value_counts()

True     15238
False     6679
Name: matched_on_program, dtype: int64

In [17]:
clean_df[[
    'awarding_agency_name',
    'matched_on_program'
]].value_counts().unstack()

matched_on_program,False,True
awarding_agency_name,,
Department of Justice,424.0,8298.0
Department of Housing and Urban Development,5479.0,6616.0
Department of Labor,424.0,NaN
Department of Education,352.0,324.0


In [18]:
clean_df[clean_df.cfda_title.str.contains('FULL-SERVICE COMMUNITY SCHOOLS')][['cfda_title', 'program_activities_funding_this_award']]

,cfda_title,program_activities_funding_this_award
18834,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0003: PROMISE NEIGHBORHOODS
24225,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0002: FULL-SERVICE COMMUNITY SCHOOLS
10632,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0002: FULL-SERVICE COMMUNITY SCHOOLS
19315,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0002: FULL-SERVICE COMMUNITY SCHOOLS
11794,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0002: FULL-SERVICE COMMUNITY SCHOOLS
...,...,...
595,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
5054,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
5020,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
4978,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS


In [19]:
clean_df[clean_df.n_prog_matches == 2][['awarding_agency_name', 'cfda_title', 'program_activities_funding_this_award']]

,awarding_agency_name,cfda_title,program_activities_funding_this_award
18834,Department of Education,"INNOVATIVE APPROACHES TO LITERACY, FULL-SERVIC...",0003: PROMISE NEIGHBORHOODS
5683,Department of Education,CONSOLIDATED GRANT TO THE OUTLYING AREAS,0001: ENGLISH LANGUAGE ACQUISITION GRANTS;0001...
4100,Department of Justice,COMMUNITY-BASED VIOLENCE PREVENTION PROGRAM,0001: CRIME VICTIMS GRANTS AND ASSISTANCE;0153...
9449,Department of Justice,EDWARD BYRNE MEMORIAL JUSTICE ASSISTANCE GRANT...,0021: STRATEGIES FOR POLICING INNOVATION (SMAR...
1507,Department of Justice,EDWARD BYRNE MEMORIAL JUSTICE ASSISTANCE GRANT...,0021: STRATEGIES FOR POLICING INNOVATION (SMAR...
...,...,...,...
595,Department of Education,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
5054,Department of Education,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
5020,Department of Education,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS
4978,Department of Education,INNOVATIVE APPROACHES TO LITERACY; PROMISE NEI...,0002: FULL-SERVICE COMMUNITY SCHOOLS


In [4]:
clean_df[
    (clean_df.n_prog_matches > 0) &
    (clean_df.awarding_agency_name == 'Department of Labor')
].to_csv('labor_grants.csv', index=False)

In [25]:
clean_df.columns

Index(['assistance_transaction_unique_key', 'assistance_award_unique_key',
       'award_id_fain', 'modification_number', 'award_id_uri', 'sai_number',
       'federal_action_obligation', 'total_obligated_amount',
       'total_outlayed_amount_for_overall_award',
       'indirect_cost_federal_share_amount',
       ...
       'initial_report_date', 'last_modified_date',
       'estimated_remaining_funds', 'spent_percent', 'remaining_funds_percent',
       'grant_is_open', 'prog_matches', 'n_prog_matches', 'matched_on_program',
       'program_match'],
      dtype='object', length=120)

In [30]:
clean_df.drop([
    'modification_number',
    'awarding_office_code',
    'funding_office_code',
    'recipient_city_code'
], axis=1).to_parquet('clean_data/clean_all_dept_all_contracts_all_cols.parquet')

In [3]:
clean_df = pd.read_parquet('clean_data/clean_all_dept_all_contracts_all_cols.parquet')

In [24]:
list(clean_df[
    (clean_df.awarding_agency_name == 'Department of Labor') &
    (clean_df.program_activities_funding_this_award.str.contains('0005: YOUTH ACTIVITIES'))
].prime_award_base_transaction_description)

['YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH BUILD',
 'YOUTH 